In [42]:
import os
import re
import json
from bs4 import BeautifulSoup

In [48]:
# 一带一路法治地图


base_dir = "/data/peitian/Data/legal/yidaiyilu"
raw_dir = os.path.join(base_dir, "raw")
output_dir = os.path.join(base_dir, "output")
os.makedirs(output_dir, exist_ok=True)

for file_path in os.listdir(raw_dir):
    name = file_path.split(".")[0]

    with open(os.path.join(raw_dir, file_path), encoding="utf-8") as f, open(os.path.join(output_dir, f"{name}.jsonl"), "w", encoding="utf-8") as g:
        soup = BeautifulSoup(f.read(), 'lxml')
        contents = soup.find("div", class_="content-detail-main").text.split("\n")

        refined_contents = []
        for c in contents:
            c = c.strip("\u3000").strip()
            if len(c):
                refined_contents.append(c)

        found_first = False
        laws = []
        for c in refined_contents:
            first_match = re.search("^(第1条)", c)
            last_match = re.search("(生效。)$", c)
            if found_first or first_match:
                found_first = True
                laws.append(c)
            if last_match:
                break

        fatiaos = []
        fatiao = []
        for law in laws:
            fatiao_match = re.search("^(第\d*条)", law)
            if fatiao_match and len(fatiao):
                concat_fatiao = "\n".join(fatiao)
                fatiaos.append(concat_fatiao)
                fatiao.clear()
            fatiao.append(law)
        concat_fatiao = "\n".join(fatiao)
        fatiaos.append(concat_fatiao)

        for fatiao in fatiaos:
            g.write(json.dumps({"text": fatiao}, ensure_ascii=False) + "\n")